# Compare Flights example

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
%matplotlib inline

## Importing libraries

In [ ]:
from rocketpy.plots.compare import CompareFlights
from rocketpy import Environment, Flight, Rocket, SolidMotor
from datetime import datetime, timedelta


## Creating the Flight objects

First, let's create the environment, motor and rocket objects.

In [ ]:
after_tomorrow = datetime.now() + timedelta(days=2)
env = Environment(latitude=-23, longitude=-49, date=after_tomorrow)
env.setAtmosphericModel(type="Forecast", file="GFS")

Pro75M1670 = SolidMotor(
    thrustSource="../../data/motors/Cesaroni_M1670.eng",
    burnOut=3.9,
    grainsCenterOfMassPosition=-0.85704,
    grainNumber=5,
    grainSeparation=5 / 1000,
    grainDensity=1815,
    grainOuterRadius=33 / 1000,
    grainInitialInnerRadius=15 / 1000,
    grainInitialHeight=120 / 1000,
    nozzleRadius=33 / 1000,
    throatRadius=11 / 1000,
    interpolationMethod="linear",
    nozzlePosition=-1.255,
    coordinateSystemOrientation="nozzleToCombustionChamber",
)

Calisto = Rocket(
    radius=127 / 2000,
    mass=19.197 - 2.956,
    inertiaI=6.60,
    inertiaZ=0.0351,
    powerOffDrag="../../data/calisto/powerOffDragCurve.csv",
    powerOnDrag="../../data/calisto/powerOnDragCurve.csv",
    centerOfDryMassPosition=0,
    coordinateSystemOrientation="tailToNose",
)

Calisto.setRailButtons(0.2, -0.5)

Calisto.addMotor(Pro75M1670, position=-1.255)

NoseCone = Calisto.addNose(length=0.55829, kind="vonKarman", position=0.71971+0.55829)

FinSet = Calisto.addTrapezoidalFins(
    n=4,
    rootChord=0.120,
    tipChord=0.040,
    span=0.100,
    position=-1.04956,
    cantAngle=0,
    radius=None,
    airfoil=None,
)

Tail = Calisto.addTail(
    topRadius=0.0635, bottomRadius=0.0435, length=0.060, position=-1.194656
)

def drogueTrigger(p, h, y):
    # p = pressure considering parachute noise signal
    # h = height above ground level considering parachute noise signal
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]

    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def mainTrigger(p, h, y):
    # p = pressure considering parachute noise signal
    # h = height above ground level considering parachute noise signal
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]

    # activate main when vz < 0 m/s and z < 800 m
    return True if y[5] < 0 and h < 800 else False


Main = Calisto.addParachute(
    "Main",
    CdS=10.0,
    trigger=mainTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue = Calisto.addParachute(
    "Drogue",
    CdS=1.0,
    trigger=drogueTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)


Now we can create different flights varying the launch angle and the rail 
inclination:

In [ ]:
inclinations = [85, 75, 60]
headings = [90, 135]
flights = []

for heading in headings:
    for inclination in inclinations:
        flight = Flight(
            environment=env,
            rocket=Calisto,
            railLength=5
            inclination=inclination,
            heading=heading,
            name=f"Incl {inclination} Head {heading}",
        )
        flights.append(flight)


In [ ]:
print("Number of flights: ", len(flights))


## Start the comparison

In [ ]:
comparison = CompareFlights(flights)


### Linear Kinematics

In [ ]:
comparison.trajectories_3d(legend=True)


In [ ]:
comparison.positions()


In [ ]:
comparison.trajectories_2d(plane= 'xy', legend=True)

In [ ]:
comparison.velocities()


In [ ]:
comparison.stream_velocities()


In [ ]:
comparison.accelerations()


### Angular kinematics

In [ ]:
comparison.angular_velocities()


In [ ]:
comparison.angular_accelerations()


### Attitudes

In [ ]:
comparison.attitude_angles()


In [ ]:
comparison.euler_angles()


In [ ]:
comparison.quaternions()


In [ ]:
comparison.angles_of_attack()


### Forces and Moments

In [ ]:
comparison.aerodynamic_forces()


In [ ]:
comparison.aerodynamic_moments()


In [ ]:
# comparison.rail_buttons_forces() # Currently not working!


### Energies and Power

In [ ]:
comparison.fluid_mechanics()


In [ ]:
comparison.energies()


In [ ]:
comparison.powers()


## Plotting everything together

Alternatively, we can plot all the results together by calling one simple 
method:

In [ ]:
# comparison.all()
